# 5. Inconsistent Data Entry

## 설명
- 사실 의미는 같으나 다르게 표현되어 다른 값으로 인식되는 데이터에 일관성을 부여하는 작업
- 기본적인 작업
    - 소문자화
    - 좌우 여백 제거
    - 중간 공백 제거(또는 공백 부여)
- `fuzzywuzzy` 패키지
    - 두 문자열이 얼마나 가까운지 확인하는 데 도움을 준다.
    - fuzzy matching: 한 문자열에서 몇 번의 변형을 거쳐야 다른 문자열이 되는지를 기준으로 가까운 정도를 나타낸다.
    - 예를 들어 in과 on는 한 번의 변화만 필요하다(i를 i로 바꾸기).

## 코드

In [7]:
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import process

In [3]:
df = pd.read_csv('data/pakistan_intellectual_capital.csv')
df.head()

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,Pakistan,2005.0,VLSI Electronics DLD Database,NaN


In [5]:
# 문제점: 독일이 Germany, germany로 쓰임, 텍스트 좌우 공백이 있음, 띄어쓰기 여부 제각각
countries = df['Country'].unique()
countries.sort()
countries

array([' Germany', ' New Zealand', ' Sweden', ' USA', 'Australia',
       'Austria', 'Canada', 'China', 'Finland', 'France', 'Greece',
       'HongKong', 'Ireland', 'Italy', 'Japan', 'Macau', 'Malaysia',
       'Mauritius', 'Netherland', 'New Zealand', 'Norway', 'Pakistan',
       'Portugal', 'Russian Federation', 'Saudi Arabia', 'Scotland',
       'Singapore', 'South Korea', 'SouthKorea', 'Spain', 'Sweden',
       'Thailand', 'Turkey', 'UK', 'USA', 'USofA', 'Urbana', 'germany'],
      dtype=object)

In [6]:
# 소문자화
df['Country'] = df['Country'].str.lower()

# 좌우 공백 제거
df['Country'] = df['Country'].str.strip()

countries = df['Country'].unique()
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea',
       'southkorea', 'spain', 'sweden', 'thailand', 'turkey', 'uk',
       'urbana', 'usa', 'usofa'], dtype=object)

In [10]:
# 'south korea'와 가장 가까운 10개 단어 추출
matches = fuzzywuzzy.process.extract("south korea", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('south korea', 100),
 ('southkorea', 48),
 ('saudi arabia', 43),
 ('norway', 35),
 ('ireland', 33),
 ('portugal', 32),
 ('singapore', 30),
 ('netherland', 29),
 ('macau', 25),
 ('usofa', 25)]

In [11]:
def replace_matches_in_column(df, column, string_to_match, min_ratio = 47):
    strings = df[column].unique()
    
    # `string_to_match`와 가장 가까운 10개 단어 추출
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # ratio가 90 이상인 텍스트만 추출
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # 추출된 텍스트에 해당하는 칼럼들의 데이터
    rows_with_matches = df[column].isin(close_matches)

    # 해당 데이터의 값을 `string_to_match`로 통일
    df.loc[rows_with_matches, column] = string_to_match

In [12]:
replace_matches_in_column(df=df, column='Country', string_to_match="south korea")

countries = df['Country'].unique()
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea', 'spain',
       'sweden', 'thailand', 'turkey', 'uk', 'urbana', 'usa', 'usofa'],
      dtype=object)